# Libraries

In [1]:
import os
import warnings

import matplotlib.pyplot as pyplot
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.metrics import (
    accuracy_score,
    classification_report,
    confusion_matrix,
    precision_score,
    r2_score,
    recall_score,
)
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.svm import LinearSVC
from sklearn.tree import DecisionTreeClassifier

warnings.filterwarnings('ignore')

In [2]:
path = '../Artifacts/postoperativedata.csv'
df = pd.read_csv(path)
df.head()

,L-CORE,L-SURF,L-O2,L-BP,SURF-STBL,CORE-STBL,BP-STBL,COMFORT,decision ADM-DECS
0,mid,low,excellent,mid,stable,stable,stable,15,A
1,mid,high,excellent,high,stable,stable,stable,10,S
2,high,low,excellent,high,stable,stable,mod-stable,10,A
3,mid,low,good,high,stable,unstable,mod-stable,15,A
4,mid,mid,excellent,high,stable,stable,stable,10,A


In [3]:
df.shape

(90, 9)

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 90 entries, 0 to 89
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   L-CORE             90 non-null     object
 1   L-SURF             90 non-null     object
 2   L-O2               90 non-null     object
 3   L-BP               90 non-null     object
 4   SURF-STBL          90 non-null     object
 5   CORE-STBL          90 non-null     object
 6   BP-STBL            90 non-null     object
 7   COMFORT            90 non-null     object
 8   decision ADM-DECS  90 non-null     object
dtypes: object(9)
memory usage: 6.5+ KB


In [5]:
for i in df.columns:
    print(df[i].unique())

['mid' 'high' 'low']
['low' 'high' 'mid']
['excellent' 'good']
['mid' 'high' 'low']
['stable' 'unstable']
['stable' 'unstable' 'mod-stable']
['stable' 'mod-stable' 'unstable']
['15' '10' '5' '7' '?']
['A' 'S' 'A ' 'I']


In [6]:
df["COMFORT"] = df["COMFORT"].replace("?", "10")

In [7]:
df["COMFORT"] = pd.to_numeric(df["COMFORT"])

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 90 entries, 0 to 89
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   L-CORE             90 non-null     object
 1   L-SURF             90 non-null     object
 2   L-O2               90 non-null     object
 3   L-BP               90 non-null     object
 4   SURF-STBL          90 non-null     object
 5   CORE-STBL          90 non-null     object
 6   BP-STBL            90 non-null     object
 7   COMFORT            90 non-null     int64 
 8   decision ADM-DECS  90 non-null     object
dtypes: int64(1), object(8)
memory usage: 6.5+ KB


In [8]:
df.isnull().sum()

L-CORE               0
L-SURF               0
L-O2                 0
L-BP                 0
SURF-STBL            0
CORE-STBL            0
BP-STBL              0
COMFORT              0
decision ADM-DECS    0
dtype: int64

In [10]:
le = LabelEncoder()
deviceEnc = pd.DataFrame(le.fit_transform(df['L-SURF']), columns=['L-SURF'])

dataEnc = pd.concat([df, deviceEnc], axis=1)
dataEnc.head()

,L-CORE,L-SURF,L-O2,L-BP,SURF-STBL,CORE-STBL,BP-STBL,COMFORT,decision ADM-DECS,L-SURF
0,mid,low,excellent,mid,stable,stable,stable,15,A,1
1,mid,high,excellent,high,stable,stable,stable,10,S,0
2,high,low,excellent,high,stable,stable,mod-stable,10,A,1
3,mid,low,good,high,stable,unstable,mod-stable,15,A,1
4,mid,mid,excellent,high,stable,stable,stable,10,A,2


In [ ]:
lightEnc = pd.DataFrame(le.fit_transform(dataEnc['L-CORE']), columns=['L-CORE'])
lightEnc.index = dataEnc.index

dataEnc2 = pd.concat([dataEnc, lightEnc], axis=1)
dataEnc2.head()

In [ ]:
sns.countplot(data=df, x="decision ADM-DECS")

# Splitting the data 

In [ ]:
X = df[:, :-1]
y = df[:, -1]

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# Model Building

In [2]:
models = {}
models["Logistic Regression"] = LogisticRegression()
models["Stochastic Gradient Descent"] = SGDClassifier(
    loss="modified_huber", shuffle=True, random_state=101
)
models["Support Vector Machines"] = LinearSVC()
models["Decision Trees"] = DecisionTreeClassifier()
models["Random Forest"] = RandomForestClassifier()
models["Naive Bayes"] = GaussianNB()
models["K-Nearest Neighbor"] = KNeighborsClassifier()

In [3]:
for i in range(len(list(models))):
    model = list(models.values())[i]
    print("Model:\t{}".format(model))

Model:	LogisticRegression()
Model:	SGDClassifier(loss='modified_huber', random_state=101)
Model:	LinearSVC()
Model:	DecisionTreeClassifier()
Model:	RandomForestClassifier()
Model:	GaussianNB()
Model:	KNeighborsClassifier()


 # Training the models and calculating the acuuaracy

In [ ]:
accuracy, precision, recall, r2 = {}, {}, {}, {}
for i in range(len(list(models))):
    model = list(models.values())[i]
    model.fit(X_train, y_train)
    predictions = model.predict(X_test)
    accuracy[key] = accuracy_score(predictions, y_test)
    precision[key] = precision_score(predictions, y_test)
    recall[key] = recall_score(predictions, y_test)
    r2[key] = r2_score(predictions, X_test)

# Accuaracy of models build 

In [ ]:
df_model = pd.DataFrame(
    index=models.keys(), columns=["Accuracy", "Precision", "Recall"]
)
df_model["Accuracy"] = accuracy.values()
df_model["Precision"] = precision.values()
df_model["Recall"] = recall.values()
df_model["R2 Score"] = r2_score.values()

df_model

In [ ]:
ax = df_model.plot.barh()
ax.legend(
    ncol=len(models.keys()), bbox_to_anchor=(0, 1), loc="lower left", prop={"size": 14}
)
plt.tight_layout()